<a href="https://colab.research.google.com/github/Chris1Areiza/lime/blob/main/Prueba_t%C3%A9cnica_MELI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# ;Lets import libraries 
import pandas as pd
from google.colab import drive

In [37]:
drive.mount('/content/drive')
data=pd.read_csv('drive/MyDrive/datameli.csv',sep=',', index_col = None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# finding shape of the dataframe
print(data.shape)

(141553, 6)


In [12]:
# Having a look at the data
print(data.head())

  event_name      item_id                     timestamp site  \
0     SEARCH          NaN  2021-08-02T23:55:38.966-0400  MLA   
1    PRODUCT  882352139.0  2021-08-02T23:55:51.673-0400  MLA   
2    PRODUCT  655266729.0  2021-08-02T23:56:16.083-0400  MLA   
3    PRODUCT  761520929.0  2021-08-02T23:56:29.989-0400  MLA   
4    PRODUCT  757586409.0  2021-08-02T23:56:47.887-0400  MLA   

                                         experiments  user_id  
0  {searchbackend/recommended-products=6157, mcli...  3204901  
1  {qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...  3204901  
2  {qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...  3204901  
3  {search/remove-ecn-tag=4954, qadb/sa-on-vip=66...  3204901  
4  {search/remove-ecn-tag=4954, qadb/sa-on-vip=66...  3204901  


In [38]:
import pandas as pd
from datetime import datetime

# convertir la columna timestamp a tipo datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# crear una columna Day con el formato YYYY-MM-DD
data['Day'] = data['timestamp'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))

# dividir la columna experiments en dos nuevas columnas: Experiment y Variant
data[['Experiment', 'Variant']] = data['experiments'].str.extractall(r'([^=,}]+)=([^=,}]+)').reset_index(drop=True)[[0, 1]]

# contar la cantidad de user_id y BUY para cada combinación de Day, Experiment y Variant
agg_data = data.groupby(['Day', 'Experiment', 'Variant']).agg({'user_id': 'nunique'}).reset_index()
buy_data = data[data['event_name'] == 'BUY'].groupby(['Day', 'Experiment', 'Variant']).agg({'event_name': 'count'}).reset_index()

# unir los dos dataframes en uno solo
final_data = pd.merge(agg_data, buy_data, on=['Day', 'Experiment', 'Variant'], how='outer').fillna(0)
final_data.columns = ['Day', 'Experiment', 'Variant', 'Users', 'Buy']

# ordenar el dataframe por Day, Experiment y Variant
final_data = final_data.sort_values(['Day', 'Experiment', 'Variant'])


In [39]:
final_data

,Day,Experiment,Variant,Users,Buy
0,2021-08-01 09,cookiesConsentBanner,DEFAULT,1,0.0
1,2021-08-01 09,frontend/assetsCdnDomainMLA,DEFAULT,1,0.0
2,2021-08-01 09,mclics/show-pads-global,5176,1,0.0
3,2021-08-01 09,mclics/show-pads-search-list,5146,1,0.0
4,2021-08-01 09,pdp/viewItemPageMigrationDesktopReviewsNoTabs,4856,1,0.0
...,...,...,...,...,...
1847,2021-08-02 23,{search/results-target-web-motors,7327,1,0.0
1848,2021-08-02 23,{search/results-target-web-motors,7328,5,0.0
1849,2021-08-02 23,{searchbackend/recommended-products,6157,28,0.0
1850,2021-08-02 23,{searchbackend/recommended-products,6158,38,2.0


In [40]:
variants_per_experiment = final_data.groupby('Experiment')['Variant'].nunique()
print(variants_per_experiment)


Experiment
 buyingflow/address_hub                               1
 buyingflow/escWebMLA                                 1
 buyingflow/secure_card                               1
 cookiesConsentBanner                                 1
 filters/sort-by-ranking                              3
 frontend/assetsCdnDomainMLA                          1
 frontend/assetsCdnDomainMLU                          1
 mclics/ads-adsearch-boost-incremental-desktop-mla    1
 mclics/search-list-algorithms                        1
 mclics/search-pads-none-desktop-mla                  2
 mclics/show-pads-global                              2
 mclics/show-pads-search-list                         2
 mshops/HideTransitionModal                           4
 pdp/cpgShowOnlyAddToCart                             1
 pdp/viewItemPageMigrationDesktopHirableSRV           1
 pdp/viewItemPageMigrationDesktopQuotableSRV          1
 pdp/viewItemPageMigrationDesktopReviewsNoTabs        2
 pdp/viewItemPageMigrationReturns    

In [41]:
import numpy as np
from scipy.stats import f_oneway, ttest_ind
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def ab_testing(data):
    unique_variants = data['Variant'].nunique()

    # Condición 1: Una variante única
    if unique_variants == 1:
        return data['Variant'].iloc[0]

    # Condición 2: Dos variantes únicas
    elif unique_variants == 2:
        variant1_data = data[data['Variant'] == data['Variant'].unique()[0]]['Buy']
        variant2_data = data[data['Variant'] == data['Variant'].unique()[1]]['Buy']
        t_stat, p_val = ttest_ind(variant1_data, variant2_data)

        if p_val < 0.05:
            return data.groupby('Variant')['Buy'].mean().idxmax()
        else:
            return 'None'

    # Condición 3: Tres o cuatro variantes únicas
    elif unique_variants in [3, 4]:
        anova_data = [data[data['Variant'] == v]['Buy'].tolist() for v in data['Variant'].unique()]
        f_stat, p_val = f_oneway(*anova_data)

        if p_val < 0.05:
            mc = pairwise_tukeyhsd(np.concatenate(anova_data), np.repeat(data['Variant'].unique(), [len(a) for a in anova_data]), alpha=0.05)
            winner_variant = None

            for i, v in enumerate(data['Variant'].unique()):
                if mc.reject[mc.groupsunique.tolist().index(v)]:
                    if winner_variant is None or data[data['Variant'] == v]['Buy'].mean() > data[data['Variant'] == winner_variant]['Buy'].mean():
                        winner_variant = v
            return winner_variant
        else:
            return 'None'

    else:
        raise ValueError("Número de variantes no soportado")

# Aplicar la función ab_testing a cada grupo de experimentos
winner_variants = final_data.groupby(['Experiment']).apply(ab_testing).reset_index()
winner_variants.columns = ['Experiment', 'winner_variant']

# Agregar la columna 'winner_variant' al dataframe 'final_data'
final_data = final_data.merge(winner_variants, on='Experiment', how='left')



/usr/local/lib/python3.9/dist-packages/scipy/stats/_stats_py.py:3895: ConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.9/dist-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: divide by zero encountered in true_divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.9/dist-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


In [42]:
final_data


,Day,Experiment,Variant,Users,Buy,winner_variant
0,2021-08-01 09,cookiesConsentBanner,DEFAULT,1,0.0,DEFAULT
1,2021-08-01 09,frontend/assetsCdnDomainMLA,DEFAULT,1,0.0,DEFAULT
2,2021-08-01 09,mclics/show-pads-global,5176,1,0.0,5176
3,2021-08-01 09,mclics/show-pads-search-list,5146,1,0.0,5146
4,2021-08-01 09,pdp/viewItemPageMigrationDesktopReviewsNoTabs,4856,1,0.0,None
...,...,...,...,...,...,...
1847,2021-08-02 23,{search/results-target-web-motors,7327,1,0.0,None
1848,2021-08-02 23,{search/results-target-web-motors,7328,5,0.0,None
1849,2021-08-02 23,{searchbackend/recommended-products,6157,28,0.0,None
1850,2021-08-02 23,{searchbackend/recommended-products,6158,38,2.0,None


In [44]:
#Instala la biblioteca de cliente de BigQuery para Python
!pip install google-cloud-bigquery


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
#Importa las bibliotecas necesarias
from google.colab import auth
from google.cloud import bigquery


In [46]:
#Autentica tu cuenta de Google en Colab
auth.authenticate_user()

In [57]:
#Variables del proyecto
project_id = 'prueba-meli-384022'
dataset_id = 'abtesting'
table_id = 'final_data'


In [58]:
#Crea una instancia del cliente de BigQuery y un objeto TableReference para la tabla de destino
client = bigquery.Client(project=project_id)
table_ref = client.dataset(dataset_id).table(table_id)

In [59]:
#Luego, crea el conjunto de datos si aún no existe
dataset = bigquery.Dataset(client.dataset(dataset_id))
try:
    client.create_dataset(dataset)
    print(f"Dataset {dataset_id} creado.")
except Exception as e:
    print(f"El dataset {dataset_id} ya existe.")

El dataset abtesting ya existe.


In [60]:
import tempfile
import pandas as pd

# Guardar el DataFrame como un archivo Parquet temporal
temp_parquet = tempfile.NamedTemporaryFile(delete=False, suffix='.parquet')
final_data.to_parquet(temp_parquet.name)

# Cargar el archivo Parquet temporal en BigQuery
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.PARQUET
job_config.autodetect = True

with open(temp_parquet.name, "rb") as source_file:
    job = client.load_table_from_file(source_file, 'prueba-meli-384022.abtesting.final_data', job_config=job_config)
job.result()

# Eliminar el archivo Parquet temporal
temp_parquet.close()



In [61]:
pip install fastapi uvicorn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [64]:
from fastapi import FastAPI, HTTPException
from typing import List
import pandas as pd
from google.cloud import bigquery

app = FastAPI()

# Establecer las credenciales del proyecto
project_id = 'prueba-meli-384022'
dataset_id = 'abtesting'
table_id = 'final_data'
client = bigquery.Client(project=project_id)

@app.get("/experiment/{experiment_id}/result")
async def read_experiment_result(experiment_id: str, day: str):
    try:
        # Consultar los resultados del experimento y día especificados
        table = client.get_table(f"{project_id}.{dataset_id}.{table_id}")
        query_job = client.query(f'''
            SELECT Experiment, Variant, Users, Buy, winner_variant
            FROM `{project_id}.{dataset_id}.{table_id}`
            WHERE Day = "{day}" AND Experiment = "{experiment_id}"
        ''')
        results = query_job.to_dataframe()

        # Si no hay resultados para el experimento y día especificados, devolver error 404
        if results.empty:
            raise HTTPException(status_code=404, detail="Experimento no encontrado")

        # Crear un diccionario con los resultados
        experiment_data = {}
        variants = []
        for i, row in results.iterrows():
            variant_data = {
                "id": row["Variant"],
                "number_of_purchases": int(row["Buy"])
            }
            variants.append(variant_data)
        experiment_data[experiment_id] = {
            "number_of_participants": int(results["Users"].sum()),
            "winner": str(results["winner_variant"].iloc[0]),
            "variants": variants
        }
        response = {"results": experiment_data}
        return response

    except Exception as e:
        # Devolver error 500 si ocurre algún error durante la consulta
        raise HTTPException(status_code=500, detail=str(e))


In [ ]:
!pip install uvicorn[standard]
!uvicorn app:app --reload


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [86177] using WatchFiles
ERROR:    Error loading ASGI app. Could not import module "app".
ERROR:    Error loading ASGI app. Could not import module "app".
ERROR:    Error loading ASGI app. Could not import module "app".
ERROR:    Error loading ASGI app. Could not import module "app".
ERROR:    Error loading ASGI app. Could not import module "app".
